In [6]:
import os
import openai
from dotenv import load_dotenv

In [4]:
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [8]:
def chat_completion()-> str:
    completion : openai.ChatCompletion = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo-1106',
        messages = [
            {'role': 'system', 'content': 'You are a poetic assistant, skilled in explaining complex programming concepts in creative poetry'},
            {'role': 'user', 'content':'Compose a poem that explains the concept of recursion in programming'}
        ]
    )
    return completion.choices[0].message.content

print(chat_completion())

In the world of code, where patterns unfold,
There lies a concept, both powerful and bold.
Recursion it's called, a magical spell,
That makes problems solve themselves so well.

Like a Russian doll, nested inside,
A function calls itself, with nowhere to hide.
The loop that repeats, without a loop,
Solving problems deep, in a graceful swoop.

Onward it travels, through layers and nodes,
Unraveling mysteries, along its roads.
A fractal dance, in the land of code,
Recursion whispers, "I'll lighten your load."

But beware, dear friend, of infinite loops,
Recursion unchecked, with no bounds to recoup.
For a stack overflow, a fate so dire,
Will crash your program in a funeral pyre.

So wield recursion wisely, with care and with grace,
In the realm of programming, it has its place.
For when used with skill, it can work wonders so grand,
Solving complex problems, with an elegant hand.
